In [1]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


import json
import random
from glob import glob

import pandas as pd
from gliner import GLiNER

In [ ]:
models_path = glob("models/checkpoint-*")
task_id = "32"
loss_max = 21.5
thr = 0.65

paths = []

for path in models_path:
    perf_file = json.load(open(os.path.join(path, "trainer_state.json"), "r"))["log_history"]
    for i in perf_file[::-1]:
        if "eval_loss" in i:
            break
    if i["eval_loss"] <= loss_max:
        paths.append(path)

paths

In [ ]:
models = [
    GLiNER.from_pretrained(path,).cuda() for path in paths
]

In [ ]:
test_data = pd.read_csv("microsoft-learn-location-mention-recognition-challenge20240905-10153-193u9hv/Test.csv")

test_data = test_data.fillna("")

# test_data["text"] = test_data["text"].apply(lambda x: " ".join(split_and_clean(x)))
test_data.sample(10)

In [ ]:
from tqdm import tqdm

label_name = ["disaster related location"]


def make_predictions(batch: list[str]):
    preds = [
        sum(rows, start=[])
        for rows in zip(
            *[
                model.batch_predict_entities(batch, label_name, threshold=0.05)
                for model in models
            ]
        )
    ]
    return preds


texts = test_data["text"].tolist()
bsize = 256

predictions = [
    make_predictions(texts[i : i + bsize])
    for i in tqdm(range(0, len(texts), bsize))
]

len(predictions)

In [ ]:
all_predictions = sum(predictions, start=[])

def drop_duplicate(places):
    return sorted(list(set(places)))
# 0.65
raws = [
	" ".join(drop_duplicate(i["text"] for i in pred if i["score"] > .7)) or " " for pred in all_predictions
]

In [ ]:
submission = test_data[["tweet_id"]]
submission["location"] = raws

submission.sample(10)

In [ ]:
submission[submission["location"] == " "]

In [9]:
submission.to_csv(f"submissions/{task_id}-GlinerLargeSplit.csv", index=False)

In [ ]:
f"submissions/{task_id}-GlinerLargeSplit.csv"